# Context Managers in Python

---

## Table of Contents
1. What are Context Managers?
2. The with Statement
3. Built-in Context Managers
4. Creating Context Managers with Classes
5. Creating Context Managers with contextlib
6. Exception Handling in Context Managers
7. Nested Context Managers
8. Async Context Managers
9. Practical Use Cases
10. Key Points
11. Practice Exercises

---

## 1. What are Context Managers?

Context managers are objects that define the runtime context to be established when executing a `with` statement.

**Purpose:**
- Resource management (files, connections, locks)
- Setup and teardown code
- Ensure cleanup happens even if exceptions occur

**Protocol:**
- `__enter__()`: Called when entering the `with` block
- `__exit__()`: Called when exiting the `with` block

In [ ]:
# Basic example - file handling
# Without context manager (old way)
f = open('test.txt', 'w')
try:
    f.write('Hello, World!')
finally:
    f.close()  # Must remember to close!

# With context manager (better way)
with open('test.txt', 'w') as f:
    f.write('Hello, World!')
# File is automatically closed, even if exception occurs

import os
os.remove('test.txt')  # Cleanup

In [ ]:
# Understanding the protocol
class MyContext:
    def __enter__(self):
        print("1. __enter__ called")
        return self  # Value assigned to 'as' variable
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("3. __exit__ called")
        # Return True to suppress exceptions
        # Return False (or None) to propagate exceptions
        return False

with MyContext() as ctx:
    print("2. Inside the with block")

print("4. After the with block")

---

## 2. The with Statement

The `with` statement ensures proper acquisition and release of resources.

In [ ]:
# How 'with' works under the hood
# This:
# with expression as variable:
#     body

# Is roughly equivalent to:
# manager = expression
# variable = manager.__enter__()
# try:
#     body
# except:
#     if not manager.__exit__(*sys.exc_info()):
#         raise
# else:
#     manager.__exit__(None, None, None)

# Demonstration
class Verbose:
    def __init__(self, name):
        self.name = name
        print(f"__init__: Creating {name}")
    
    def __enter__(self):
        print(f"__enter__: Entering {self.name}")
        return f"Resource: {self.name}"
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"__exit__: Exiting {self.name}")
        print(f"  Exception type: {exc_type}")
        print(f"  Exception value: {exc_val}")
        return False

print("Before with:")
with Verbose("test") as resource:
    print(f"Inside with: {resource}")
print("After with:")

In [ ]:
# The 'as' clause is optional
class NoReturn:
    def __enter__(self):
        print("Entering...")
        # Returns None implicitly
    
    def __exit__(self, *args):
        print("Exiting...")

with NoReturn():
    print("Inside")

# Can still use 'as' but it will be None
with NoReturn() as x:
    print(f"x = {x}")

---

## 3. Built-in Context Managers

In [ ]:
# File objects
with open('example.txt', 'w') as f:
    f.write('Line 1\n')
    f.write('Line 2\n')

with open('example.txt', 'r') as f:
    content = f.read()
    print(content)

import os
os.remove('example.txt')

In [ ]:
# Threading locks
import threading

lock = threading.Lock()

# Without context manager
lock.acquire()
try:
    # Critical section
    print("Lock acquired (manual)")
finally:
    lock.release()

# With context manager
with lock:
    # Critical section
    print("Lock acquired (context manager)")

In [ ]:
# decimal context for precision
from decimal import Decimal, localcontext

print(f"Default precision: {Decimal(1) / Decimal(7)}")

with localcontext() as ctx:
    ctx.prec = 50
    print(f"High precision: {Decimal(1) / Decimal(7)}")

print(f"Back to default: {Decimal(1) / Decimal(7)}")

In [ ]:
# Suppressing exceptions with contextlib.suppress
from contextlib import suppress

# Instead of:
try:
    os.remove('nonexistent.txt')
except FileNotFoundError:
    pass

# Use:
with suppress(FileNotFoundError):
    os.remove('nonexistent.txt')
    print("This won't print if file doesn't exist")

print("Continues here")

In [ ]:
# Redirecting output
from contextlib import redirect_stdout, redirect_stderr
import io

# Capture stdout to a string
buffer = io.StringIO()
with redirect_stdout(buffer):
    print("This goes to buffer")
    print("So does this")

captured = buffer.getvalue()
print(f"Captured: {captured!r}")

---

## 4. Creating Context Managers with Classes

In [ ]:
# Timer context manager
import time

class Timer:
    """Context manager to time code execution."""
    
    def __init__(self, label=""):
        self.label = label
        self.elapsed = None
    
    def __enter__(self):
        self.start = time.perf_counter()
        return self
    
    def __exit__(self, *args):
        self.elapsed = time.perf_counter() - self.start
        label = f" ({self.label})" if self.label else ""
        print(f"Elapsed{label}: {self.elapsed:.4f}s")
        return False

with Timer("sleep test"):
    time.sleep(0.1)

with Timer("computation") as t:
    sum(range(1000000))

print(f"Accessed later: {t.elapsed:.4f}s")

In [ ]:
# Database connection context manager
class DatabaseConnection:
    """Simulated database connection manager."""
    
    def __init__(self, connection_string):
        self.connection_string = connection_string
        self.connection = None
    
    def __enter__(self):
        print(f"Connecting to {self.connection_string}...")
        self.connection = {"status": "connected", "db": self.connection_string}
        return self.connection
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("Closing connection...")
        self.connection = None
        if exc_type is not None:
            print(f"Error occurred: {exc_val}")
        return False  # Don't suppress exceptions

with DatabaseConnection("postgres://localhost/mydb") as conn:
    print(f"Using connection: {conn}")

print("Connection is now closed")

In [ ]:
# Temporary directory changer
import os

class ChangeDirectory:
    """Temporarily change working directory."""
    
    def __init__(self, new_path):
        self.new_path = new_path
        self.saved_path = None
    
    def __enter__(self):
        self.saved_path = os.getcwd()
        os.chdir(self.new_path)
        return self
    
    def __exit__(self, *args):
        os.chdir(self.saved_path)
        return False

print(f"Current directory: {os.getcwd()}")

with ChangeDirectory('..'):
    print(f"Changed to: {os.getcwd()}")

print(f"Back to: {os.getcwd()}")

In [ ]:
# Transaction-like context manager
class Transaction:
    """Simulate database transaction with commit/rollback."""
    
    def __init__(self, data):
        self.data = data
        self.backup = None
    
    def __enter__(self):
        # Save current state
        self.backup = self.data.copy()
        print("Transaction started")
        return self.data
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is not None:
            # Rollback on error
            print(f"Rolling back due to: {exc_val}")
            self.data.clear()
            self.data.update(self.backup)
        else:
            # Commit
            print("Transaction committed")
        return True  # Suppress exceptions

data = {"balance": 100}

with Transaction(data) as d:
    d["balance"] += 50
    print(f"During transaction: {d}")

print(f"After successful transaction: {data}")

with Transaction(data) as d:
    d["balance"] -= 200
    raise ValueError("Insufficient funds!")

print(f"After failed transaction: {data}")

---

## 5. Creating Context Managers with contextlib

In [ ]:
# @contextmanager decorator - generator-based context managers
from contextlib import contextmanager

@contextmanager
def timer(label=""):
    """Time code execution using generator-based context manager."""
    start = time.perf_counter()
    try:
        yield  # Control passes to 'with' block
    finally:
        elapsed = time.perf_counter() - start
        print(f"{label}: {elapsed:.4f}s")

with timer("List comprehension"):
    [x**2 for x in range(100000)]

In [ ]:
# Yielding a value
@contextmanager
def open_file(path, mode='r'):
    """Custom file context manager."""
    f = open(path, mode)
    try:
        yield f  # This is the value bound to 'as'
    finally:
        f.close()
        print(f"File {path} closed")

with open_file('test.txt', 'w') as f:
    f.write('Hello!')

os.remove('test.txt')

In [ ]:
# Temporary environment variable
@contextmanager
def temp_env_var(key, value):
    """Temporarily set an environment variable."""
    old_value = os.environ.get(key)
    os.environ[key] = value
    try:
        yield
    finally:
        if old_value is None:
            del os.environ[key]
        else:
            os.environ[key] = old_value

print(f"DEBUG before: {os.environ.get('DEBUG', 'not set')}")

with temp_env_var('DEBUG', 'true'):
    print(f"DEBUG inside: {os.environ.get('DEBUG')}")

print(f"DEBUG after: {os.environ.get('DEBUG', 'not set')}")

In [ ]:
# Working directory changer using contextmanager
@contextmanager
def working_directory(path):
    """Temporarily change working directory."""
    old_cwd = os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(old_cwd)

print(f"Before: {os.getcwd()}")
with working_directory('..'):
    print(f"During: {os.getcwd()}")
print(f"After: {os.getcwd()}")

In [ ]:
# contextlib.closing - for objects with close() method
from contextlib import closing
from urllib.request import urlopen

# Example with a class that has close() but isn't a context manager
class Resource:
    def __init__(self, name):
        self.name = name
        print(f"Opened {name}")
    
    def use(self):
        print(f"Using {self.name}")
    
    def close(self):
        print(f"Closed {self.name}")

# Without context manager - might forget to close
r = Resource("manual")
r.use()
r.close()

# With closing() - guaranteed to close
with closing(Resource("auto")) as r:
    r.use()

In [ ]:
# ExitStack for dynamic context management
from contextlib import ExitStack

# Useful when you don't know how many context managers you need
filenames = ['file1.txt', 'file2.txt', 'file3.txt']

with ExitStack() as stack:
    files = [
        stack.enter_context(open(fn, 'w'))
        for fn in filenames
    ]
    for i, f in enumerate(files):
        f.write(f'Content for file {i}\n')

# All files are now closed
print("All files written and closed")

# Cleanup
for fn in filenames:
    os.remove(fn)

---

## 6. Exception Handling in Context Managers

In [ ]:
# __exit__ receives exception info
class ExceptionDemo:
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"exc_type: {exc_type}")
        print(f"exc_val: {exc_val}")
        print(f"exc_tb: {exc_tb}")
        return False  # Propagate exception

# Normal exit
print("=== Normal exit ===")
with ExceptionDemo():
    pass

# Exception exit
print("\n=== Exception exit ===")
try:
    with ExceptionDemo():
        raise ValueError("Something went wrong!")
except ValueError:
    print("Exception was propagated")

In [ ]:
# Suppressing exceptions
class SuppressException:
    def __init__(self, *exception_types):
        self.exception_types = exception_types
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is not None:
            if issubclass(exc_type, self.exception_types):
                print(f"Suppressed: {exc_type.__name__}: {exc_val}")
                return True  # Suppress
        return False  # Propagate

with SuppressException(ValueError, TypeError):
    raise ValueError("This will be suppressed")
print("Continued after suppressed exception")

try:
    with SuppressException(ValueError):
        raise KeyError("This will NOT be suppressed")
except KeyError:
    print("KeyError was propagated")

In [ ]:
# Exception handling with @contextmanager
@contextmanager
def error_handler():
    """Handle and log errors."""
    try:
        yield
    except Exception as e:
        print(f"Error caught: {type(e).__name__}: {e}")
        # Re-raise if you don't want to suppress
        # raise

with error_handler():
    raise RuntimeError("Something failed!")

print("Execution continues")

In [ ]:
# Cleanup always runs
@contextmanager
def guaranteed_cleanup():
    print("Setup")
    try:
        yield
    except Exception as e:
        print(f"Exception: {e}")
        raise  # Re-raise the exception
    finally:
        print("Cleanup (always runs)")

try:
    with guaranteed_cleanup():
        raise ValueError("Error!")
except ValueError:
    print("Exception handled outside")

---

## 7. Nested Context Managers

In [ ]:
# Multiple context managers in one 'with'
@contextmanager
def context(name):
    print(f"Enter {name}")
    try:
        yield name
    finally:
        print(f"Exit {name}")

# Nested (indented)
print("=== Nested ===")
with context("A"):
    with context("B"):
        with context("C"):
            print("Inside all")

In [ ]:
# Multiple on same line (Python 3.1+)
print("=== Same line ===")
with context("A"), context("B"), context("C"):
    print("Inside all")

In [ ]:
# Parenthesized (Python 3.10+)
print("=== Parenthesized ===")
with (
    context("A") as a,
    context("B") as b,
    context("C") as c,
):
    print(f"Values: {a}, {b}, {c}")

In [ ]:
# Using ExitStack for dynamic nesting
from contextlib import ExitStack

contexts_needed = ["DB", "Cache", "File"]

with ExitStack() as stack:
    # Dynamically enter contexts
    values = [stack.enter_context(context(name)) for name in contexts_needed]
    print(f"All contexts entered: {values}")

In [ ]:
# ExitStack with callbacks
from contextlib import ExitStack

def cleanup(name):
    print(f"Cleaning up {name}")

with ExitStack() as stack:
    # Register cleanup callbacks (LIFO order)
    stack.callback(cleanup, "first")
    stack.callback(cleanup, "second")
    stack.callback(cleanup, "third")
    print("All callbacks registered")

---

## 8. Async Context Managers

In [ ]:
# Async context manager using class
import asyncio

class AsyncTimer:
    """Async context manager for timing async operations."""
    
    async def __aenter__(self):
        self.start = asyncio.get_event_loop().time()
        print("Timer started")
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        elapsed = asyncio.get_event_loop().time() - self.start
        print(f"Elapsed: {elapsed:.4f}s")
        return False

async def main():
    async with AsyncTimer():
        await asyncio.sleep(0.1)
        print("Async operation done")

await main()

In [ ]:
# Async context manager using asynccontextmanager
from contextlib import asynccontextmanager

@asynccontextmanager
async def async_resource(name):
    """Async resource management."""
    print(f"Acquiring {name}...")
    await asyncio.sleep(0.05)  # Simulate async acquisition
    try:
        yield name
    finally:
        await asyncio.sleep(0.05)  # Simulate async cleanup
        print(f"Released {name}")

async def main():
    async with async_resource("database") as db:
        print(f"Using {db}")

await main()

In [ ]:
# Async connection pool simulation
@asynccontextmanager
async def connection_pool(size=3):
    """Simulated async connection pool."""
    connections = []
    
    # Create connections
    for i in range(size):
        await asyncio.sleep(0.01)  # Simulate connection time
        connections.append(f"conn_{i}")
    
    print(f"Pool created with {size} connections")
    
    try:
        yield connections
    finally:
        # Close all connections
        for conn in connections:
            await asyncio.sleep(0.01)  # Simulate close time
        print(f"Pool closed, {len(connections)} connections released")

async def main():
    async with connection_pool(5) as pool:
        print(f"Available connections: {pool}")

await main()

---

## 9. Practical Use Cases

In [ ]:
# 1. Logging context
@contextmanager
def log_context(operation):
    """Log operation start and end."""
    import logging
    logger = logging.getLogger(__name__)
    
    print(f"Starting: {operation}")
    start = time.time()
    try:
        yield
        print(f"Completed: {operation} in {time.time() - start:.2f}s")
    except Exception as e:
        print(f"Failed: {operation} with {e}")
        raise

with log_context("data processing"):
    time.sleep(0.1)
    # process data...

In [ ]:
# 2. Temporary file creation
import tempfile

@contextmanager
def temp_file(suffix="", prefix="tmp", dir=None, delete=True):
    """Create a temporary file that's cleaned up after use."""
    fd, path = tempfile.mkstemp(suffix=suffix, prefix=prefix, dir=dir)
    try:
        os.close(fd)
        yield path
    finally:
        if delete and os.path.exists(path):
            os.remove(path)

with temp_file(suffix=".txt") as path:
    print(f"Temp file: {path}")
    with open(path, 'w') as f:
        f.write("Temporary content")

print(f"File exists after: {os.path.exists(path)}")

In [ ]:
# 3. Atomic file write
@contextmanager
def atomic_write(filepath, mode='w'):
    """Write to file atomically (all or nothing)."""
    temp_path = filepath + '.tmp'
    try:
        with open(temp_path, mode) as f:
            yield f
        # If we get here, write was successful
        os.replace(temp_path, filepath)
    except:
        # On any error, remove temp file
        if os.path.exists(temp_path):
            os.remove(temp_path)
        raise

# Successful write
with atomic_write('atomic_test.txt') as f:
    f.write('Line 1\n')
    f.write('Line 2\n')

with open('atomic_test.txt') as f:
    print(f"Content: {f.read()}")

os.remove('atomic_test.txt')

In [ ]:
# 4. Indentation for pretty printing
class Indenter:
    """Context manager for nested indentation."""
    
    def __init__(self, indent_str="  "):
        self.indent_str = indent_str
        self.level = 0
    
    def __enter__(self):
        self.level += 1
        return self
    
    def __exit__(self, *args):
        self.level -= 1
        return False
    
    def print(self, text):
        print(self.indent_str * self.level + text)

indent = Indenter()
indent.print("Root")

with indent:
    indent.print("Child 1")
    with indent:
        indent.print("Grandchild 1.1")
        indent.print("Grandchild 1.2")
    indent.print("Child 2")

indent.print("Back to root")

In [ ]:
# 5. Mock patching (simplified version of unittest.mock.patch)
@contextmanager
def patch_attribute(obj, attr, new_value):
    """Temporarily replace an attribute."""
    original = getattr(obj, attr)
    setattr(obj, attr, new_value)
    try:
        yield
    finally:
        setattr(obj, attr, original)

class Config:
    debug = False
    api_url = "https://api.production.com"

print(f"Before: debug={Config.debug}, url={Config.api_url}")

with patch_attribute(Config, 'debug', True), \
     patch_attribute(Config, 'api_url', 'http://localhost:8000'):
    print(f"During: debug={Config.debug}, url={Config.api_url}")

print(f"After: debug={Config.debug}, url={Config.api_url}")

In [ ]:
# 6. Performance profiler
import cProfile
import pstats
from io import StringIO

@contextmanager
def profile(sort_by='cumulative', lines=10):
    """Profile code execution."""
    profiler = cProfile.Profile()
    profiler.enable()
    try:
        yield
    finally:
        profiler.disable()
        stream = StringIO()
        stats = pstats.Stats(profiler, stream=stream)
        stats.sort_stats(sort_by)
        stats.print_stats(lines)
        print(stream.getvalue())

with profile(lines=5):
    result = sum(x**2 for x in range(10000))

---

## 10. Key Points

1. **Protocol**: `__enter__` (setup) and `__exit__` (teardown)
2. **Guarantee**: Cleanup runs even if exceptions occur
3. **Return Value**: `__enter__` returns value for `as` clause
4. **Exception Handling**: `__exit__` can suppress by returning `True`
5. **@contextmanager**: Simpler way using generators
6. **ExitStack**: Manage dynamic number of contexts
7. **Async**: Use `__aenter__`/`__aexit__` or `@asynccontextmanager`
8. **Nesting**: Multiple contexts can be combined
9. **Built-ins**: files, locks, suppress, redirect_stdout
10. **Use Cases**: Resources, transactions, temporary state

---

## 11. Practice Exercises

In [ ]:
# Exercise 1: Create a context manager that suppresses specific exceptions
# and logs them to a list
# - Should suppress specified exception types
# - Should store (exception_type, message) tuples in a list
# - Should have a 'exceptions' attribute to access logged exceptions

class ExceptionLogger:
    pass

# Test:
# logger = ExceptionLogger(ValueError, TypeError)
# with logger:
#     raise ValueError("test error")
# print(logger.exceptions)  # [(ValueError, 'test error')]

In [ ]:
# Exercise 2: Create a rate limiter context manager
# - Limits how many times a block can execute per second
# - Blocks (sleeps) if rate would be exceeded
# - Use @contextmanager decorator

@contextmanager
def rate_limit(calls_per_second):
    pass

# Test:
# for i in range(5):
#     with rate_limit(2):  # Max 2 calls per second
#         print(f"Call {i} at {time.time():.2f}")

In [ ]:
# Exercise 3: Create a context manager that tracks nested execution depth
# - Should work correctly with nested 'with' statements
# - Should provide current depth and max depth reached
# - Class-based implementation

class DepthTracker:
    pass

# Test:
# tracker = DepthTracker()
# with tracker:  # depth = 1
#     with tracker:  # depth = 2
#         with tracker:  # depth = 3
#             print(f"Current depth: {tracker.current_depth}")  # 3
#         print(f"Current depth: {tracker.current_depth}")  # 2
# print(f"Max depth: {tracker.max_depth}")  # 3

In [ ]:
# Exercise 4: Create a context manager for HTML tag generation
# - Should support any tag name and attributes
# - Should support nested tags
# - Should build HTML string that can be retrieved

class HTMLBuilder:
    pass

# Test:
# html = HTMLBuilder()
# with html.tag('div', class_='container'):
#     with html.tag('h1'):
#         html.text('Title')
#     with html.tag('p'):
#         html.text('Paragraph')
# print(html.render())
# Output: <div class="container"><h1>Title</h1><p>Paragraph</p></div>

In [ ]:
# Exercise 5: Create an async context manager for connection pooling
# - Pool has limited connections
# - acquire() waits if pool is empty
# - release() returns connection to pool

class AsyncConnectionPool:
    pass

# Test:
# async def main():
#     pool = AsyncConnectionPool(max_connections=2)
#     async with pool.acquire() as conn1:
#         print(f"Got {conn1}")
#         async with pool.acquire() as conn2:
#             print(f"Got {conn2}")

---

## Solutions

In [ ]:
# Solution 1:
class ExceptionLogger:
    def __init__(self, *exception_types):
        self.exception_types = exception_types
        self.exceptions = []
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is not None:
            if issubclass(exc_type, self.exception_types):
                self.exceptions.append((exc_type, str(exc_val)))
                return True
        return False

logger = ExceptionLogger(ValueError, TypeError)

with logger:
    raise ValueError("first error")

with logger:
    raise TypeError("second error")

print(f"Logged exceptions: {logger.exceptions}")

In [ ]:
# Solution 2:
import time
from contextlib import contextmanager

# Store state outside the function for persistence across calls
_rate_limit_state = {}

@contextmanager
def rate_limit(calls_per_second, key="default"):
    """Limit execution rate."""
    min_interval = 1.0 / calls_per_second
    
    if key not in _rate_limit_state:
        _rate_limit_state[key] = 0
    
    last_call = _rate_limit_state[key]
    now = time.time()
    
    if last_call > 0:
        elapsed = now - last_call
        if elapsed < min_interval:
            time.sleep(min_interval - elapsed)
    
    _rate_limit_state[key] = time.time()
    yield

print("Rate limited calls (2/sec):")
start = time.time()
for i in range(5):
    with rate_limit(2):
        print(f"Call {i} at {time.time() - start:.2f}s")

In [ ]:
# Solution 3:
class DepthTracker:
    def __init__(self):
        self.current_depth = 0
        self.max_depth = 0
    
    def __enter__(self):
        self.current_depth += 1
        self.max_depth = max(self.max_depth, self.current_depth)
        return self
    
    def __exit__(self, *args):
        self.current_depth -= 1
        return False

tracker = DepthTracker()

with tracker:
    print(f"Depth: {tracker.current_depth}")
    with tracker:
        print(f"Depth: {tracker.current_depth}")
        with tracker:
            print(f"Depth: {tracker.current_depth}")
        print(f"Depth: {tracker.current_depth}")
    print(f"Depth: {tracker.current_depth}")

print(f"Max depth reached: {tracker.max_depth}")

In [ ]:
# Solution 4:
class HTMLBuilder:
    def __init__(self):
        self._parts = []
    
    @contextmanager
    def tag(self, name, **attrs):
        # Build opening tag with attributes
        attr_str = ""
        for key, value in attrs.items():
            # Handle class_ -> class conversion
            key = key.rstrip('_')
            attr_str += f' {key}="{value}"'
        
        self._parts.append(f"<{name}{attr_str}>")
        try:
            yield
        finally:
            self._parts.append(f"</{name}>")
    
    def text(self, content):
        self._parts.append(content)
    
    def render(self):
        return "".join(self._parts)

html = HTMLBuilder()
with html.tag('div', class_='container'):
    with html.tag('h1'):
        html.text('Title')
    with html.tag('p', id='intro'):
        html.text('This is a paragraph.')

print(html.render())

In [ ]:
# Solution 5:
import asyncio
from contextlib import asynccontextmanager

class AsyncConnectionPool:
    def __init__(self, max_connections=5):
        self.max_connections = max_connections
        self._pool = asyncio.Queue()
        self._created = 0
    
    async def _create_connection(self):
        self._created += 1
        conn_id = self._created
        await asyncio.sleep(0.01)  # Simulate connection time
        return f"Connection_{conn_id}"
    
    @asynccontextmanager
    async def acquire(self):
        # Try to get from pool, or create new if under limit
        if self._pool.empty() and self._created < self.max_connections:
            conn = await self._create_connection()
        else:
            conn = await self._pool.get()
        
        try:
            yield conn
        finally:
            await self._pool.put(conn)

async def main():
    pool = AsyncConnectionPool(max_connections=2)
    
    async with pool.acquire() as conn1:
        print(f"Got {conn1}")
        async with pool.acquire() as conn2:
            print(f"Got {conn2}")
        print(f"Released conn2")
    print(f"Released conn1")
    
    # Reuse from pool
    async with pool.acquire() as conn3:
        print(f"Reused {conn3}")

await main()